In [ ]:
!mv /content/*.pcd /content/pcd

In [ ]:
pip install open3d

In [ ]:
import open3d

In [ ]:
import os
print(os.getcwd())
os.chdir('pcd')
print(os.getcwd())

/content
/content/pcd


In [ ]:
# pcd 파일들이 D:/project/2023/lidar/python/pcd 폴드에 있는 경우
!python /content/mergepcd3.py /content/pcd /content/out1203.pcd

In [ ]:
!python /content/pcdtolas.py /content/out1203.pcd /content/out1203.las

In [ ]:
# pip install laspy

In [ ]:
import laspy
import os

def append_to_las(in_las, out_las):
    with laspy.open(out_las, mode='a') as outlas:
        with laspy.open(in_las) as inlas:
            for points in inlas.chunk_iterator(2_000_000):
                outlas.append_points(points)

# Specify the directory containing your input LAS files
inDir = 'out'

# Specify the path to the output LAS file
out_las = 'merged/m1_32.las'

# Append the points from each LAS file to the output file
for (dirpath, dirnames, filenames) in os.walk(inDir):
    for inFile in filenames:
        if inFile.endswith('.las'):
            in_las = os.path.join(dirpath, inFile)
            append_to_las(in_las, out_las)

In [ ]:
import laspy
import numpy as np

def rotate_point(point, angle_degrees):
    # Rotate a 2D point around the origin
    x, y = point
    angle_radians = np.radians(angle_degrees)
    x_rotated = x * np.cos(angle_radians) - y * np.sin(angle_radians)
    y_rotated = x * np.sin(angle_radians) + y * np.cos(angle_radians)
    return x_rotated, y_rotated

# Read a LAS file
#las_file = laspy.read("merged/m1_32.las")
las_file = laspy.read("/content/out1203.las")

# Access the x, y, z coordinates of the points
x = las_file.x
y = las_file.y
z = las_file.z

# Rotate the coordinates by degrees
for i in range(len(las_file.points)):
    x[i], y[i] = rotate_point((x[i], y[i]), 90)

# Modify the values if needed
# For example, you could add 10 to all x coordinates
las_file.x += 478161
las_file.y += 3965697
las_file.z += 72

# Save the modified LAS file
las_file.write("m1_32_utm_1203_1.las")

In [ ]:
pwd

'/content/pcd'

In [ ]:
import laspy

# Read a LAS file
las_file = laspy.read("merged/m1_32.las")

# Access the x, y, z coordinates of the points
x = las_file.x
y = las_file.y
z = las_file.z

# Modify the values if needed
# For example, you could add 10 to all x coordinates
las_file.x += 478114
las_file.y += 3965734
las_file.z += 72

# Save the modified LAS file
las_file.write("merged/m1_32_utm.las")

In [ ]:
import open3d as o3d
import numpy as np

N=1000
#pcd = o3d.io.read_point_cloud('d:/project/2023/lidar/data/output_merged.pcd')
pcd = o3d.io.read_point_cloud('/content/out1203.pcd')
#pcd = o3d.io.read_point_cloud('d:/project/2023/lidar/data/a1.pcd')
# fit to unit cube
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
o3d.visualization.draw_geometries([pcd])

In [ ]:
import open3d as o3d
import numpy as np

N=1000
#pcd = o3d.io.read_point_cloud('d:/project/2023/lidar/data/output_merged.pcd')
pcd = o3d.io.read_point_cloud('merged/out1203.pcd')
#pcd = o3d.io.read_point_cloud('d:/project/2023/lidar/data/a1.pcd')
# fit to unit cube
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
o3d.visualization.draw_geometries([pcd])

In [ ]:
import open3d as o3d
import pandas as pd
import plotly.express as px
# Convert point cloud to a Pandas DataFrame
point_cloud_np = pd.DataFrame(np.asarray(pcd.points), columns=['X', 'Y', 'Z'])

# Plot the point cloud using Plotly
fig = px.scatter_3d(point_cloud_np, x='X', y='Y', z='Z', opacity=0.6)
fig.update_layout(title='Point Cloud Visualization')
fig.show()


In [ ]:
print('octree division')
octree = o3d.geometry.Octree(max_depth=6)
octree.convert_from_point_cloud(pcd, size_expand=0.01)
o3d.visualization.draw_geometries([octree])

In [ ]:
print('octree division')
octree = o3d.geometry.Octree(max_depth=8)
octree.create_from_voxel_grid(voxel_grid)
o3d.visualization.draw_geometries([octree])

In [ ]:
print('voxelization')
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,
                                                            voxel_size=0.001)
o3d.visualization.draw_geometries([voxel_grid])

In [ ]:
grid_size = voxel_grid.get_max_bound() - voxel_grid.get_min_bound()
print(f"Voxel Grid Size: {grid_size}")